<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_GPT_finetuning_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter
## (3 )Finetuning de GPT 4 mini para clasificación binaria

**Natalia Dedandi**



La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction


Guia para finetuning:

https://medium.com/@garethcull/fine-tuning-ai-models-a-practical-guide-for-beginners-dc313b2e0f76

In [1]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install datasets seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Fo

In [3]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [4]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Creo una función para que GPT 4 determine si es odioso o no el TW

Basado en este ejemplo:
https://community.openai.com/t/i-am-facing-issue-in-running-sentiment-analysis-api-with-openai-while-i-have-fullfilled-all-the-basic-code-which-i-got-from-google-bard/570554


In [ ]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


In [ ]:
##creo una columna donde paso el 1 a HATEFUL  y el 0 a NO HATEFUL


In [ ]:
def format_for_fine_tuning(data_tweet):
  # Specify the columns
  id_column = 'id'
  text_column = 'text'
  hate = 'text'

  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {
          'custom_id': str(row[id_column]),
          'method': 'POST',
          'url': '/v1/chat/completions',
          'body':{
              'model': 'gpt-4o-mini',
              'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row[text_column]},
                  {'role': 'assistant', 'content': row[text_column]}
                  ],
              'max_tokens': 20,  # Limit response to a single word
              'temperature': 0  # Keep response consistent
          }
      }
      for _, row in df.iterrows()
  ]
  return(batch)

In [ ]:
training_tweets=format_for_fine_tuning(df_train_sample)
training_tweets

In [ ]:
#{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
def format_for_fine_tuning(data_tweet):

    training_data_list = []

    data_tweet = data_tweet.reset_index()  # make sure indexes pair with number of rows

    for index, row in data_tweet.iterrows():
        updated_data ={
            "prompt": row['prompt'],
            "completion": row['completion']}
        training_data_list.append(updated_data)

    return training_data_list